# Représentation des données : types et valeurs de base - les flottants


## Les nombres flottants en Python

Les nombres flottants n'ont pas grand chose à voir avec les réels ! Il sont à comparer plutôt avec les nombres en notation scientifique des calculatrices, mais en base 2.

La mise en oeuvre des nombres flottants en Python repose sur la norme IEEE 754 double précision, qui spécifie qu'un nombre flottant est représenté par :
* son signe **s** sur 1 bit,
* sa mantisse **m** sur 52 bits en binaire, normalisée sous la forme $1.b_1b_2...b_{52}$ où chaque $b_i$ est noté 0 ou 1 (le premier 1 n'est en fait pas représenté)
* son exposant **e** en puissance de 2 représenté sur 11 bits en binaire (exposants -1022 à 1023)

Le nombre flottant représenté est : $s . m * 2^e$

La notation des nombres flottants en base 2, étend la notation binaire des entiers. Chaque bit $b_i$ de la mantisse vaut $2^{-i}$.

**Exemple** : `1.001` est le code de $1 + 1/8$ ou $1.125$.

* **Activité** : explorer les limites de représentation des nombres flottants en terme de précision. Essayer d'interpréter la précision relative disponible par rapport au nombre de bits de la mantisse.

In [ ]:
(1.5000000000000000, 1.5000000000000001, 1.5000000000000002)

Ces exemples montrent la limite de précision de la mantisse : 52 chiffres significatifs en base 2, qui correspondent à 16 chiffres significatifs en base 10. La précision relative est en fait de $1/2^{52}$ soit environ $2.10^{−16}$.

Remarque : cette valeur est mémorisée en Python dans le module `sys` sous le nom `sys.float_info.epsilon`.

In [ ]:
import sys
sys.float_info.epsilon

* **Activité** : explorer les limites de précision des calculs sur les nombres flottants. Est-ce que tous les nombres décimaux peuvent être représentés exactement par des flottants ?

In [ ]:
0.25 + 0.5

In [ ]:
0.1 + 0.2

**Analyse** :

Pour analyser cette approximation sur le calcul avec des nombres décimaux, il faut examiner en détail le codage en binaire à virgule flottante de la mantisse.

> $1.b_1b_2...b_{52}$

Les nombres pouvant être codés de manière exacte sous cette forme sont les nombres pouvant s'écrire comme une somme : 
$1 + \Sigma_{i=1}^{52}(b_i/2^i)$
C'est à dire les multiples entiers de $1/2^{52}$, ce qui n'est pas le cas de $0.1$ ni de $0.2$ d'où l'approximation effectuée lors de leur représentation en machine et lors de l'addition.

* **Activité :** Calculer à la main les quelques premiers chiffres de la représentation en base 2 du nombre 0,1.

**Propagation des erreurs de représentation** :

L'erreur relative effectuée lors de la représentation, peut être amplifiée si on soustrait deux nombres proches.

In [ ]:
1023.1 - 1023

### Nombres représentables

Le plus grand nombre flottant représentable  au format normalisé est :

$$1,11111...111 \times 2^{1023} = (2 - 2^{-52}) \times 2^{1023} = 2^{1024} - 2^{971} \approx 1.7976931348623157.10^{308}$$

Le plus petit nombre flottant strictement positif représentable au format normalisé  est :

$$1,0 \times 2^{-1022} = 2^{-1022} \approx 2.2250738585072014.10^{-308}$$

In [ ]:
sys.float_info.max

In [ ]:
sys.float_info.min

### Expérimentation avec affichage du codage des flottants en binaire

Pour observer les codages utilisés, on utilise la fonction Python `float.hex` qui montre le codage hexadécimal d'un nombre flottant.

Pour obtenir un résultat plus facilement interprétable, on propose une fonction `float2bin` qui donne la représentation binaire d'un flottant à partir de sa représentation héxadécimale.


In [ ]:
def hex2bin(h):
    """Convertit un chiffre hexadécimal '0' à 'f' en binaire sur 4 bits"""
    n = int(h, base=16)
    b = ''
    for i in range(4):
        b = str(n % 2) + b
        n = n // 2
    return(b)

def float2bin(f):
    """Retourne la représentation interne en binaire d'un float strictement positif"""
    s = float.hex(f)
    b = '1.'
    for i in range(4,17):
        b = b + hex2bin(s[i])
    b = b + s[17:]
    return(b)

**Précision de la représentation**

In [ ]:
(float2bin(1.5000000000000000), float2bin(1.5000000000000001), float2bin(1.5000000000000002))

A la lecture de leur représentation, on comprend pourquoi il n'y a pas de flottant entre $1.5$ et $1.5000000000000002$.
Le nombre $1.5000000000000001$ est représenté comme $1.5$.

La précision possible avec une mantisse de 52 bits se situe au niveau du dernier bit qui vaut $2^{-52}$ soit environ $2.10^{−16}$.

**Approximation de la représentation**

Tous les décimaux n'ont pas une représentation exacte en base 2 virgule flottante. Si on observe la représentation de 1.1 ou 1.2, on constate que la représentation cyclique est tronquée.

In [ ]:
(float2bin(1.1), float2bin(1.2))


Le codage de $1.1$ montre une représentation binaire à virgule `1.000 1100 1100 1100 1100 ...` L'écriture exacte en binaire à virgule est ici cyclique et infinie, ce qui explique l'approximation effectuée quand l'écriture est tronquée à 52 bits. 

### Conversion de binaire en flottant

Pour poursuivre les explorations, on peut se doter des fonctions réciproques pour convertir une représentation binaire, vers le nombre flottant correspondant. On utilise pour cela la fonction Python `float.fromhex` 

In [ ]:
def bin2hex(b):
    """Convertit un nombre binaire sur 4 bits en un chiffre hexadécimal"""
    h = hex(int(b, base=2))
    return(h[2])

def bin2float(b):
    """Retourne le flottant correspondant à une représentation interne en binaire"""
    h = '0x1.'
    for i in range (2,54,4):
        h = h + bin2hex(b[i:i+4])
    h = h + b[54:]
    return(float.fromhex(h))

On peut ainsi rechercher le plus petit flottant strictement supérieur à 1.0

In [ ]:
bin2float("1.0000000000000000000000000000000000000000000000000001p+0")

le plus grand strictement inférieur à 2.0

In [ ]:
bin2float("1.1111111111111111111111111111111111111111111111111111p+0")

Le premier flottant strictement supérieur à $2^{53} = 9007199254740992$

In [ ]:
bin2float("1.0000000000000000000000000000000000000000000000000001p+53")

Ce dernier calcul donne le nombre $2^{53}+2$. 

A partir de $2^{52}$, tous les flottants sont des entiers.

A partir de $2^{53}$, tous les entiers ne sont pas représentables en tant que flottants.

* **Activité** : observer en détail la répartition des nombres flottants

Entre nombres positifs et négatifs, entre petits et grands nombres. 


On a approximativement :

* 1/4 des flottants inférieurs à -1 : signe négatif, exposant positif ou nul
* 1/4 des flottants entre -1 et 0 : signe négatif, exposant négatif
* 1/4 des flottants entre 0 et 1: signe positif, exposant négatif
* 1/4 des flottants supérieurs à 1: signe positif, exposant positif ou nul

A noter que la norme IEEE 754 permet aussi de représenter, par des combinaisons réservées des valeurs d'exposant et de mantisse, les valeurs NaN (not a number), $+\infty$, $-\infty$.

### Comparaisons entre flottants

A cause des multiples approximations effectuées lors de la représentation ou du calcul sur les flottants, il ne faut pas tester l'égalité de flottants.

Ceci ferait courir le risque de programmes qui ne s'arrêtent pas si une boucle contient de tels tests. 

In [ ]:
0.1 + 0.2 == 0.3

In [ ]:
from math import sqrt
sqrt(2.0)**2 == 2.0

In [ ]:
1 + sys.float_info.epsilon / 2 == 1.0

In [ ]:
x  = float(2**53)
x + 1 == x

### Conclusion

Les nombres flottants sont tout à fait adaptés en précision et en portée aux calculs de physique de l'infiniment petit à l'infiniment grand, mais ils ne correspondent pas du tout aux nombres réels définis en mathématiques.

Equipe pédagoqique DIU EIL, ressource éducative libre distribuée sous [Licence Creative Commons Attribution - Pas d’Utilisation Commerciale - Partage dans les Mêmes Conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/) ![Licence Creative Commons](https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png)